### **Segunda limpieza del dataset: [01_ventas.xls](https://docs.google.com/spreadsheets/d/1NFH-6nRnqv7jqEjHuLQpdWG9Z-PhsvnT/edit?usp=drive_link&ouid=100767986945074374198&rtpof=true&sd=true)**

- 1. Reconfigurar PKs (idProducto) de PRODUCTO
- 2. Mapear DETALLE_VENTA/producto con PRODUCTO/idProducto
- 3. Modificar formato VENTA/hora

---

#### Importar excel (*01_ventas.xls*) para trabajar sobre los atributos desde Pandas

In [1]:
import pandas as pd

# Leer archivos .xls <pip install xlrd>
df_venta = pd.read_excel('01_ventas.xls', sheet_name="VENTA")
df_producto = pd.read_excel('01_ventas.xls', sheet_name="PRODUCTO")
df_detalle_venta = pd.read_excel('01_ventas.xls', sheet_name="DETALLE_VENTA")

---

#### 1. Reconfigurar PKs (*idProducto*) de PRODUCTO:
- Prefijo: primeras tres letras de la columna *categoria* en mayúscula.
- Sufijo: valor autoincremental (001, 002, ...) para cada producto dentro de la categoría.

In [2]:
print(df_producto[["idProducto", "nombre", "categoria"]].head())

   idProducto           nombre categoria
0           1        Medialuna    Dulces
1           2            Latte     Cafés
2           3            Chipá   Salados
3           4       Flat White     Cafés
4           5  Onda Medialunas    Promos


In [3]:
# Generar IDs de 3 cifras por cada productos dentro de una categoría
df_producto["id_nuevo"] = (
    df_producto.groupby("categoria").cumcount() + 1
).astype(str).str.zfill(3)

# Generar prefijo en mayúscula con las primeras 3 letras de la categoría
df_producto["id_nuevo"] = df_producto["categoria"].str[:3].str.upper() + df_producto["id_nuevo"]

# Cambiar los valores de idProducto por los nuevos
df_producto["idProducto"] = df_producto["id_nuevo"]
df_producto = df_producto.drop(columns=["id_nuevo"])

print(df_producto[["idProducto", "nombre", "categoria"]].head(10))

  idProducto             nombre categoria
0     DUL001          Medialuna    Dulces
1     CAF001              Latte     Cafés
2     SAL001              Chipá   Salados
3     CAF002         Flat White     Cafés
4     PRO001    Onda Medialunas    Promos
5     DUL002  Cookie Red Velvet    Dulces
6     DUL003     Cookie Clásica    Dulces
7     DUL004   Pan de Chocolate    Dulces
8     CAF003              Magic     Cafés
9     DUL005          Croissant    Dulces


---

#### 2. Mapear *DETALLE_VENTA/producto* con *PRODUCTO/idProducto*:

- Eliminar información redundante.
- Relacionar ambas tablas.

In [4]:
print(df_detalle_venta[["idVenta","producto","categoria"]].head())

   idVenta          producto       categoria
0     6812             Latte  Café con leche
1     6812  Pan de Chocolate          Dulces
2     6812         Vaso 12oz     Adicionales
3     6811    Cookie Clásica         Cookies
4     6811        Alfacookie       Alfajores


In [5]:
# Crear mapa de nombres a IDs
mapa = dict(zip(df_producto["nombre"], df_producto["idProducto"]))

# Reemplazar en detalle_venta
df_detalle_venta["idProducto"] = df_detalle_venta["producto"].map(mapa)

# Eliminar columnas reemplazadas
df_detalle_venta = df_detalle_venta.drop(columns=["producto", "categoria"])

# Ordenar columnas
df_detalle_venta = df_detalle_venta[["idVenta", "idProducto", "cantidad", "precio", "costo", "cancelada"]]

print(df_detalle_venta[["idVenta", "idProducto"]].head())

   idVenta idProducto
0     6812     CAF001
1     6812     DUL004
2     6812     ADI001
3     6811     DUL003
4     6811     DUL009


---

#### 3. Modificar formato VENTA/hora
- Actual: dia/mes/año hh:mm:ss
- Nuevo: hh:mm

In [6]:
print(df_venta[["idVenta", "fecha", "hora"]].head())

   idVenta      fecha                hora
0     6812 2025-09-04 2025-09-04 19:14:06
1     6811 2025-09-04 2025-09-04 19:11:49
2     6810 2025-09-04 2025-09-04 19:10:58
3     6809 2025-09-04 2025-09-04 19:07:29
4     6808 2025-09-04 2025-09-04 19:04:18


In [7]:
# Establecer formato datetime
df_venta["hora"] = pd.to_datetime(df_venta["hora"], format="%d/%m/%Y %H:%M:%S")

# Cambiar formato a "hh:mm"
df_venta["hora"] = df_venta["hora"].dt.strftime("%H:%M")

print(df_venta[["idVenta", "fecha", "hora"]].head())

   idVenta      fecha   hora
0     6812 2025-09-04  19:14
1     6811 2025-09-04  19:11
2     6810 2025-09-04  19:10
3     6809 2025-09-04  19:07
4     6808 2025-09-04  19:04
